# LFT 06 — Spacetime Test: Direct $\mathbb{R}^4$ Embeddings of $\Pi_5$ (N=6)

This notebook tests **direct 4D embeddings** of the 5D permutohedron $\Pi_5$ (N=6) to separate **space (3 axes)** and **time (1 axis)** in a way that aligns with LFT's **L-flow** (monotone descent of inversions). We implement and compare three embeddings:

1. **PCA(5→4)** — optimal linear projection for squared error (baseline).
2. **Flow-aligned time + orthogonal 3D space** — choose a global time axis that best aligns with the local descent field; pick space as top-3 PCs in the orthogonal complement.
3. **Constraint-optimized 4D** — small convex optimization to balance (i) low edge distortion and (ii) strong time-alignment monotonicity.

We report metrics:
- **Global stress** (variance loss), **edge distortion** (adjacent edges), and **pairwise RMS error (sampled)**.
- **Time alignment**: cosine between the time axis and the descent field; **monotonicity** of $h$ along the time coordinate.

**Outcome**: A direct **spacetime factorization** (3+1) that matches LFT: space = geometric rank (A$_5$ roots), time = global L-flow direction.

## 0. Utilities: Π₅ vertices, adjacent graph, inversions, and local descent field
We build the 5D coordinates in the sum-zero space $V\subset\mathbb{R}^6$, the adjacent-edge graph, and define:
- **h(perm)** = inversion count.
- **Local descent vector** at a vertex = mean of (neighbor − vertex) over edges that **reduce** h.
This yields a vector field $D\in\mathbb{R}^{720\times 5}$ to define/learn a global time axis.

In [ ]:
import numpy as np, itertools, networkx as nx

def sum_zero_basis(N):
    diffs = np.zeros((N, N-1))
    for i in range(N-1):
        diffs[i, i] = 1.0
        diffs[i+1, i] = -1.0
    U,S,Vt = np.linalg.svd(diffs, full_matrices=False)
    return U  # N x (N-1)

def permutohedron_coords(N):
    B = sum_zero_basis(N)
    a = np.arange(N, dtype=float) - (N-1)/2.0
    perms = list(itertools.permutations(range(N)))
    Vcoords = np.zeros((len(perms), N-1))
    for k, p in enumerate(perms):
        v = a[list(p)]
        Vcoords[k] = B.T @ v
    return Vcoords, perms

def cayley_adjacent_graph(N, perms):
    idx = {p:i for i,p in enumerate(perms)}
    G = nx.Graph()
    G.add_nodes_from(range(len(perms)))
    gens = [(i, i+1) for i in range(N-1)]
    for p in perms:
        u = idx[p]
        for (i,j) in gens:
            q = list(p)
            q[i], q[j] = q[j], q[i]
            v = idx[tuple(q)]
            if u < v:
                G.add_edge(u, v)
    return G

def inversion_count_tuple(p):
    inv=0
    for i in range(len(p)):
        for j in range(i+1, len(p)):
            if p[i] > p[j]:
                inv += 1
    return inv

def local_descent_field(V, perms, G):
    # For each vertex u, average vectors to neighbors v that reduce h
    n = V.shape[0]
    D = np.zeros_like(V)
    for u in range(n):
        pu = perms[u]
        hu = inversion_count_tuple(pu)
        vecs=[]
        for v in G.neighbors(u):
            pv = perms[v]
            hv = inversion_count_tuple(pv)
            if hv < hu:  # descent
                vecs.append(V[v]-V[u])
        if vecs:
            D[u] = np.mean(np.vstack(vecs), axis=0)
    return D

V6, perms6 = permutohedron_coords(6)
G6 = cayley_adjacent_graph(6, perms6)
D6 = local_descent_field(V6, perms6, G6)
V6_mean = V6.mean(axis=0, keepdims=True)
print({'nodes': V6.shape[0], 'edges': G6.number_of_edges(), 'nonzero_descent_vectors': int((np.linalg.norm(D6, axis=1)>0).sum())})

## 1. Baseline: PCA(5→4)
Compute the PCA projection to 4D (optimal linear map for squared error) and record **retained variance** and **edge distortion**. This is our baseline "no spacetime separation" embedding.

In [ ]:
def pca_project(X, k):
    Xc = X - X.mean(axis=0, keepdims=True)
    U,S,Vt = np.linalg.svd(Xc, full_matrices=False)
    return Xc @ Vt[:k].T, Vt[:k], (S[:k]**2).sum()/(S**2).sum()

X4_pca, Vt4_pca, retained_pca = pca_project(V6, 4)
print({'retained_variance_PCA4': float(retained_pca)})

## 2. Flow-aligned time axis + orthogonal 3D space
We compute a **global time direction** \(w_t\in\mathbb{R}^5\) as the first right singular vector of the descent field **D** (after centering on nonzero rows). Then:
- Project data onto **time coordinate**: \(t = V w_t\).
- Build an orthonormal basis **W_s** for the 3D **spatial subspace** in the orthogonal complement of \(w_t\) by taking the top-3 PCs of \(V\) after deflating \(w_t\).
- The 4D embedding is **Y = [V W_s, t]**.

We report **time-alignment** (cosine with local descent vectors), **monotonicity** of \(h\) along \(t\), and edge/global errors.

In [ ]:
def flow_time_axis(D):
    mask = (np.linalg.norm(D, axis=1) > 0)
    Dnz = D[mask]
    Dnz = Dnz - Dnz.mean(axis=0, keepdims=True)
    U,S,Vt = np.linalg.svd(Dnz, full_matrices=False)
    w_t = Vt[0]
    w_t = w_t/np.linalg.norm(w_t)
    return w_t

def orth_spatial_basis(V, w_t, k=3):
    # Deflate w_t, compute top-k PCs in orthogonal complement
    Wt = w_t.reshape(-1,1)
    P_t = Wt @ Wt.T  # projector onto time axis
    V_defl = V - (V @ P_t)  # remove time component
    Vc = V_defl - V_defl.mean(axis=0, keepdims=True)
    U,S,Vt = np.linalg.svd(Vc, full_matrices=False)
    W_s = Vt[:k].T  # (5 x k)
    # Orthonormalize against w_t just in case
    # Gram-Schmidt
    Q = []
    for i in range(W_s.shape[1]):
        v = W_s[:,i]
        v = v - (v@w_t)*w_t
        for q in Q:
            v = v - (v@q)*q
        v = v/np.linalg.norm(v)
        Q.append(v)
    W_s = np.stack(Q, axis=1)
    return W_s

def embed_flow_spacetime(V, D):
    w_t = flow_time_axis(D)
    W_s = orth_spatial_basis(V, w_t, k=3)
    Y_space = V @ W_s  # (n x 3)
    t = V @ w_t        # (n,)
    Y4 = np.hstack([Y_space, t.reshape(-1,1)])
    return Y4, W_s, w_t

Y4_flow, W_s, w_t = embed_flow_spacetime(V6, D6)
print('||w_t|| =', np.linalg.norm(w_t))

### Time-alignment and monotonicity metrics
- **Alignment**: For each vertex with nonzero descent vector \(d_u\), compute cosine \(\cos\theta_u = \frac{\langle d_u, w_t\rangle}{\|d_u\|}\).
- **Monotonicity**: For each adjacent edge that reduces \(h\), check if the **time coordinate decreases** along the edge (consistent orientation). Report fraction of descent edges with \(\Delta t < 0\) after fixing orientation so that identity has minimal time (or we can set sign by requiring mean descent cosine > 0).

In [ ]:
def time_alignment_metrics(V, D, w_t, perms, G):
    # Align sign so that average descent has negative time (decreasing t is decreasing h)
    mask = (np.linalg.norm(D, axis=1) > 0)
    mean_cos = np.mean([(D[i]@w_t)/np.linalg.norm(D[i]) for i in range(len(D)) if mask[i]])
    if mean_cos > 0:
        w_use = w_t
        sign = +1
    else:
        w_use = -w_t
        sign = -1
    # Cosine stats
    cosines = np.array([(D[i]@w_use)/np.linalg.norm(D[i]) for i in range(len(D)) if mask[i]])
    # Monotonicity on descent edges
    tcoord = V @ w_use
    good = 0; total = 0
    for u,v in G.edges():
        hu = inversion_count_tuple(perms[u]); hv = inversion_count_tuple(perms[v])
        if hv < hu:
            total += 1
            if tcoord[v] < tcoord[u]:
                good += 1
        elif hu < hv:
            total += 1
            if tcoord[u] < tcoord[v]:
                good += 1
    frac_mono = good/total if total>0 else 1.0
    return {
        'sign': int(sign),
        'mean_cos_descent': float(np.mean(cosines)),
        'median_cos_descent': float(np.median(cosines)),
        'q25_cos_descent': float(np.quantile(cosines, 0.25)),
        'q75_cos_descent': float(np.quantile(cosines, 0.75)),
        'frac_mono_edges': float(frac_mono),
        'descent_edges_count': int(total)
    }

align_flow = time_alignment_metrics(V6, D6, w_t, perms6, G6)
align_flow

### Edge/global error metrics for flow-aligned embedding
We compute **edge distortions** and **variance retention** for the flow-aligned 4D embedding and compare to PCA.

In [ ]:
import pandas as pd, matplotlib.pyplot as plt, os
os.makedirs('./outputs', exist_ok=True)

def edge_errors(V, Y4, G):
    rows=[]
    for u,v in G.edges():
        l5 = np.linalg.norm(V[u]-V[v])
        l4 = np.linalg.norm(Y4[u]-Y4[v])
        rel = abs(l4-l5)/l5 if l5>0 else 0.0
        rows.append(rel)
    arr = np.array(rows)
    return {
        'mean': float(arr.mean()),
        'median': float(np.median(arr)),
        'q25': float(np.quantile(arr,0.25)),
        'q75': float(np.quantile(arr,0.75)),
        'max': float(arr.max())
    }, arr

def retained_variance_linear(Y4, V):
    # Fit least-squares linear map from V to Y4 and compute R^2 (variance explained)
    Vc = V - V.mean(axis=0, keepdims=True)
    Yc = Y4 - Y4.mean(axis=0, keepdims=True)
    W, *_ = np.linalg.lstsq(Vc, Yc, rcond=None)
    Yhat = Vc @ W
    ss_res = np.sum((Yc-Yhat)**2)
    ss_tot = np.sum(Yc**2)
    return 1.0 - ss_res/ss_tot

edge_flow_stats, edge_flow = edge_errors(V6, Y4_flow, G6)
R2_flow = retained_variance_linear(Y4_flow, V6)
edge_pca_stats, edge_pca = edge_errors(V6, X4_pca, G6)
R2_pca = retained_variance_linear(X4_pca, V6)
print({'edge_flow':edge_flow_stats,'edge_pca':edge_pca_stats,'R2_flow':float(R2_flow),'R2_pca':float(R2_pca)})

## 3. Constraint-optimized 4D: balancing edge fidelity and time monotonicity
We solve a small ridge-regularized least-squares for a **5×4** projection matrix **W** that balances:

- (A) **Edge fidelity**: for each adjacent edge (u,v), \(\|W^T(V_u-V_v)\|-\|V_u-V_v\|\) small.
- (B) **Time monotonicity**: define time as the 4th column of W; for descent edges, require \(\langle W_t^T(V_v-V_u), 1\rangle < 0\) (soft with hinge penalty).

We implement a simple projected gradient descent with column-orthonormalization (QR) to keep W stable. This is exploratory but often yields improvements in monotonicity with minimal loss of edge fidelity.

In [ ]:
def optimize_W(V, perms, G, steps=200, lr=1e-2, lam_edge=1.0, lam_time=1.0, seed=0):
    rng = np.random.default_rng(seed)
    # init from flow-aligned space+time basis
    # W: 5x4 with orthonormal columns approximately
    # first 3 columns = W_s, last = w_t
    W = np.hstack([W_s, w_t.reshape(-1,1)])
    # loss components
    def loss_and_grad(W):
        W = W.reshape(5,4)
        # Edge fidelity term
        L_edge = 0.0
        G_edge = np.zeros_like(W)
        for u,v in G.edges():
            dv = (V[v]-V[u]).reshape(1,-1)  # 1x5
            d5 = np.linalg.norm(dv)
            y = dv @ W      # 1x4
            d4 = np.linalg.norm(y)
            if d5>0 and d4>0:
                err = (d4 - d5)
                L_edge += 0.5*err*err
                # grad wrt W: (err/d4) * y^T * dv
                G_edge += (err/d4) * (dv.T @ y)
        # Time hinge term
        w_t_curr = W[:,3]
        L_time = 0.0
        G_time = np.zeros_like(W)
        tcoord = V @ w_t_curr
        for u,v in G.edges():
            hu = inversion_count_tuple(perms[u]); hv = inversion_count_tuple(perms[v])
            if hv == hu: continue
            # require: if hv<hu then t[v] < t[u]; hinge on margin m = (t[v]-t[u])
            if hv < hu:
                m = tcoord[v] - tcoord[u]
                if m >= 0:
                    L_time += m*m*0.5
                    # grad wrt w_t only: d/dw m = V[v]-V[u]
                    G_time[:,3] += m*(V[v]-V[u])
            else:  # hu < hv
                m = tcoord[u] - tcoord[v]
                if m >= 0:
                    L_time += m*m*0.5
                    G_time[:,3] += m*(V[u]-V[v])
        L = lam_edge*L_edge + lam_time*L_time
        G = lam_edge*G_edge + lam_time*G_time
        return L, G
    
    for it in range(steps):
        L, G = loss_and_grad(W)
        W = W - lr*G
        # re-orthonormalize columns (QR)
        Q, R = np.linalg.qr(W)
        W = Q
        if (it+1)%50==0:
            print({'iter':it+1, 'loss': float(L)})
    return W

W_opt = optimize_W(V6, perms6, G6, steps=200, lr=5e-3, lam_edge=1.0, lam_time=2.0, seed=42)
Y4_opt = V6 @ W_opt
edge_opt_stats, edge_opt = edge_errors(V6, Y4_opt, G6)
R2_opt = retained_variance_linear(Y4_opt, V6)
align_opt = time_alignment_metrics(V6, D6, W_opt[:,3], perms6, G6)
print({'edge_opt':edge_opt_stats, 'R2_opt':float(R2_opt), 'align_opt':align_opt})

## 4. Summary & Manuscript-Ready Captions

**Findings (to be filled by the run):**
- **PCA(5→4)**: retained variance = …; edge distortion mean/median/IQR/max = …
- **Flow-aligned 3+1D**: retained variance (R²) = …; edge distortion = …; **mean descent cosine** = …; **fraction of descent edges monotone in t** = …
- **Constraint-optimized 3+1D**: retained variance = …; edge distortion = …; **descent alignment/monotonicity** = … (often improved over PCA with small cost).

### Captions
- **Fig. ST-1.** *Cosine distribution between local descent vectors and the global time axis for the flow-aligned embedding.*
- **Fig. ST-2.** *Histogram of relative edge-length errors for PCA and flow-aligned embeddings.*
- **Fig. ST-3.** *Trade-off curve for edge distortion vs. descent-edge monotonicity across optimized embeddings (varying λ_time).*

## 5. Plots — Cosine alignment, edge-error comparison, and λ_time sweep
These figures make the spacetime factorization quantitative and visually clear:
- **Cosine histogram** between local descent vectors and the learned global time axis.
- **Edge error histograms** comparing PCA(5→4) vs. Flow-aligned 3+1D.
- **λ_time sweep** trade-off curve showing edge fidelity vs. descent-edge monotonicity.

In [ ]:
import numpy as np, matplotlib.pyplot as plt, os
os.makedirs('./outputs', exist_ok=True)

# 5.1 Cosine alignment histogram
mask = (np.linalg.norm(D6, axis=1) > 0)
cosines = np.array([(D6[i]@w_t)/np.linalg.norm(D6[i]) for i in range(len(D6)) if mask[i]])
plt.figure()
plt.hist(cosines, bins=40)
plt.xlabel('cos(angle) between local descent d_u and global time axis w_t')
plt.ylabel('Count')
plt.title('Flow-aligned time: descent–time cosine distribution (N=6)')
plt.tight_layout()
plt.savefig('./outputs/N6_descent_time_cosine_hist.png', dpi=150)
plt.close()
print('Saved ./outputs/N6_descent_time_cosine_hist.png')

In [ ]:
# 5.2 Edge error comparison histogram: PCA vs Flow-aligned 3+1D
plt.figure()
plt.hist(edge_pca, bins=40, alpha=0.5, label='PCA 5→4')
plt.hist(edge_flow, bins=40, alpha=0.5, label='Flow-aligned 3+1D')
plt.xlabel('Relative edge-length error')
plt.ylabel('Count')
plt.title('Edge distortion: PCA vs Flow-aligned (N=6, adjacent edges)')
plt.legend()
plt.tight_layout()
plt.savefig('./outputs/N6_edge_error_hist_PCA_vs_Flow.png', dpi=150)
plt.close()
print('Saved ./outputs/N6_edge_error_hist_PCA_vs_Flow.png')

In [ ]:
# 5.3 λ_time sweep: trade-off between edge fidelity and descent monotonicity
lam_values = [0.0, 0.5, 1.0, 2.0, 4.0]
trade = []
for lam in lam_values:
    W_try = optimize_W(V6, perms6, G6, steps=120, lr=5e-3, lam_edge=1.0, lam_time=lam, seed=123)
    Y_try = V6 @ W_try
    edge_stats, edge_arr = edge_errors(V6, Y_try, G6)
    align_stats = time_alignment_metrics(V6, D6, W_try[:,3], perms6, G6)
    trade.append((lam, edge_stats['mean'], align_stats['frac_mono_edges']))
    print({'lam_time':lam, 'edge_mean':edge_stats['mean'], 'frac_mono':align_stats['frac_mono_edges']})

trade = np.array(trade, dtype=float)
plt.figure()
plt.plot(trade[:,0], trade[:,1], marker='o')
plt.xlabel('λ_time (monotonicity weight)')
plt.ylabel('Mean relative edge error')
plt.title('Edge fidelity vs λ_time')
plt.tight_layout()
plt.savefig('./outputs/N6_trade_edge_vs_lambda.png', dpi=150)
plt.close()

plt.figure()
plt.plot(trade[:,0], trade[:,2], marker='o')
plt.xlabel('λ_time (monotonicity weight)')
plt.ylabel('Fraction of descent edges with Δt < 0')
plt.title('Monotonicity vs λ_time')
plt.tight_layout()
plt.savefig('./outputs/N6_trade_mono_vs_lambda.png', dpi=150)
plt.close()
print('Saved sweep plots: N6_trade_edge_vs_lambda.png and N6_trade_mono_vs_lambda.png')